# Utils

> Utility functions for portfolio analysis

In [ ]:
#| default_exp utils

In [ ]:
#| export
#| echo: false
import io
import inspect
from typing import Union,TypeVar
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
import hypersheets.stats as stats


In [ ]:
#| export
def mtd(df):
    """Restrict a dataframe to only month to date"""
    return df[df.index >= dt.datetime.now(
    ).strftime('%Y-%m-01')]

In [ ]:
#| export
def qtd(df):
    """Restrict a dataframe to only quarter to date (quarters starting in Jan, Apr, Jun, Oct) """
    return df[df.index >= dt.datetime.now(
    ).strftime('%Y-%m-01')]

In [ ]:
#| export
def ytd(df):
    """Restrict a dataframe to only year to date"""
    return df[df.index >= dt.datetime.now(
    ).strftime('%Y-01-01')]

In [ ]:
#| hide


In [ ]:
#| export
def pandas_date(df, dates):
    """Filters a dataframe (with date as the index), to its values on specific days"""
    if not isinstance(dates, list):
        dates = [dates]
    return df[df.index.isin(dates)]

In [ ]:
#| eval: false
df = pd.DataFrame({
    'name': ['alice','bob','charlie','ryan'],
    'DoB': ['2005-10-15','2002-09-03','2001-01-01','1999-12-31']
})
df = df.set_index('DoB')
dates = ['2001-01-01','2005-10-15']
pandas_date(df, dates)

,name
DoB,
2005-10-15,alice
2001-01-01,charlie


In [ ]:
#| export
def pandas_current_month(df):
    """an alternative method to mtd. remove?"""
    n = dt.datetime.now()
    daterange = pd.date_range(dt.date(n.year, n.month, 1), n)
    return df[df.index.isin(daterange)]

In [ ]:
#| export
def multi_shift(df, shift=3):
    """Get last N rows relative to another row in dataframe of values, with a sorted index"""
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df)

    dfs = [df.shift(i) for i in np.arange(shift)]
    for ix, dfi in enumerate(dfs[1:]):
        dfs[ix + 1].columns = [str(col) for col in dfi.columns + str(ix + 1)]
    return pd.concat(dfs, axis = 1, sort=True)

In [ ]:
#| eval: false
df = pd.DataFrame({
    'value': [10,15,13,7,12,6],
    'Date': ['1999-10-15','1999-10-16','1999-10-17','1999-10-18','1999-10-19','1999-10-20']
})
df = df.set_index('Date')
multi_shift(df,3)

,value,value1,value2
Date,,,
1999-10-15,10,NaN,NaN
1999-10-16,15,10.0,NaN
1999-10-17,13,15.0,10.0
1999-10-18,7,13.0,15.0
1999-10-19,12,7.0,13.0
1999-10-20,6,12.0,7.0


In [ ]:
#| export
def to_excess_returns(returns:Union['~pd.Series', '~pd.DataFrame'], # Returns
 rf:Union[float, '~pd.Series', '~pd.DataFrame'] , # Risk-Free rate(s)
 nperiods:int=None # Will convert rf to different frequency using deannualize
 )->Union['~pd.Series', '~pd.DataFrame']: #Returns - risk free rate
    """
    Calculates excess returns by subtracting
    risk-free returns from total returns
    """
    if isinstance(rf, int):
        rf = float(rf)

    if not isinstance(rf, float):
        rf = rf[rf.index.isin(returns.index)]

    if nperiods is not None:
        # deannualize
        rf = np.power(1 + rf, 1. / nperiods) - 1.

    return returns - rf

In [ ]:
#| export
def prepare_returns(data, rf=0., nperiods=None):
    """Converts price data into returns + cleanup"""
    data = data.copy()
    function = inspect.stack()[1][3]
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            if data[col].dropna().min() >= 0 and data[col].dropna().max() > 1:
                data[col] = data[col].pct_change()
    elif data.min() >= 0 and data.max() > 1:
        data = data.pct_change()

    # cleanup data
    data = data.replace([np.inf, -np.inf], float('NaN'))

    if isinstance(data, (pd.DataFrame, pd.Series)):
        data = data.fillna(0).replace(
            [np.inf, -np.inf], float('NaN'))
    unnecessary_function_calls = ['_prepare_benchmark',
                                  'cagr',
                                  'gain_to_pain_ratio',
                                  'rolling_volatility']


    if function not in unnecessary_function_calls:
        if rf > 0:
            return to_excess_returns(data, rf, nperiods)
    return data

In [ ]:
#| export
def to_returns(prices, rf=0.):
    """Calculates the simple arithmetic returns of a price series"""
    return prepare_returns(prices, rf)

In [ ]:
#| export
def to_prices(returns, base=1e5):
    """Converts returns series to price data"""
    returns = returns.copy().fillna(0).replace(
        [np.inf, -np.inf], float('NaN'))

    return base + base * stats.compsum(returns)

In [ ]:
#| eval: false
# mock returns array
data = np.array([0.1, -0.2, 0.25, 0.5, -0.8])
returns = pd.Series(data)

In [ ]:
#| eval: false
to_prices(returns, base=100)

0    110.0
1     88.0
2    110.0
3    165.0
4     33.0
dtype: float64

In [ ]:
#| export
def to_log_returns(returns, rf=0., nperiods=None):
    """Converts returns series to log returns"""
    returns = prepare_returns(returns, rf, nperiods)
    try:
        return np.log(returns+1).replace([np.inf, -np.inf], float('NaN'))
    except Exception:
        return 0.

The log return is defined as $lr_{i}=\log\frac{p_{i}}{p_{i-1}}$ where $lr_{i}$ and $p_{i}$ are the log returns and price on day $i$ respectively

In [ ]:
#| export
def log_returns(returns, rf=0., nperiods=None):
    """Shorthand for to_log_returns"""
    return to_log_returns(returns, rf, nperiods)

In [ ]:
#| export
def exponential_stdev(returns, window=30, is_halflife=False):
    """Returns series representing exponential volatility of returns"""
    returns = prepare_returns(returns)
    halflife = window if is_halflife else None
    return returns.ewm(com=None, span=window,
                       halflife=halflife, min_periods=window).std()

In [ ]:
#| export
def download_prices(ticker, period="max"):
    """download daily adjusted close prices from yahoo"""
    if isinstance(period, pd.DatetimeIndex):
        p = {"start": period[0]}
    else:
        p = {"period": period}

    return yf.Ticker(ticker).history(**p)['Close']# this is automatically the adjusted value

In [ ]:
#| eval: false
download_prices('SPY',  period="5y").tail(10)

Date
2022-09-15    388.524017
2022-09-16    385.559998
2022-09-19    388.549988
2022-09-20    384.089996
2022-09-21    377.390015
2022-09-22    374.220001
2022-09-23    367.950012
2022-09-26    364.309998
2022-09-27    363.380005
2022-09-28    370.529999
Name: Close, dtype: float64

In [ ]:
#| export
def download_returns(ticker, period="max"):
    """download returns from yahoo"""
    if isinstance(period, pd.DatetimeIndex):
        p = {"start": period[0]}
    else:
        p = {"period": period}

    return prepare_returns(yf.Ticker(ticker).history(**p)['Close']) # this is automatically the adjusted value

In [ ]:
#| eval: false
download_returns('SPY',  period="5y").tail(10)

Date
2022-09-15   -0.011353
2022-09-16   -0.007629
2022-09-19    0.007755
2022-09-20   -0.011479
2022-09-21   -0.017444
2022-09-22   -0.008400
2022-09-23   -0.016755
2022-09-26   -0.009893
2022-09-27   -0.002553
2022-09-28    0.019676
Name: Close, dtype: float64

In [ ]:
#| export
def prepare_benchmark(benchmark=None, period="max", rf=0.,
                       prep_returns=True):
    """
    Fetch benchmark if ticker is provided, and pass through
    prepare_returns()
    period can be options or (expected) pd.DatetimeIndex range
    """
    if benchmark is None:
        return None

    if isinstance(benchmark, str):
        benchmark = download_returns(benchmark)

    elif isinstance(benchmark, pd.DataFrame):
        benchmark = benchmark[benchmark.columns[0]].copy()

    if isinstance(period, pd.DatetimeIndex) \
        and set(period) != set(benchmark.index):

        # Adjust Benchmark to Strategy frequency
        benchmark_prices = to_prices(benchmark, base=1)
        new_index = pd.date_range(start=period[0], end=period[-1], freq='D')
        benchmark = benchmark_prices.reindex(new_index, method='bfill') \
            .reindex(period).pct_change().fillna(0)
        benchmark = benchmark[benchmark.index.isin(period)]

    if prep_returns:
        return prepare_returns(benchmark.dropna(), rf=rf)
    return benchmark.dropna()

In [ ]:
#| eval: false
prepare_benchmark('SPY',  period="5y").tail(10)

Date
2022-09-15   -0.011353
2022-09-16   -0.007629
2022-09-19    0.007755
2022-09-20   -0.011479
2022-09-21   -0.017444
2022-09-22   -0.008400
2022-09-23   -0.016755
2022-09-26   -0.009893
2022-09-27   -0.002553
2022-09-28    0.019676
Name: Close, dtype: float64

In [ ]:
#| export
def rebase(prices, base=100.):
    """
    Rebase all series to a given intial base.
    This makes comparing/plotting different series together easier.
    Args:
        * prices: Expects a price series/dataframe
        * base (number): starting value for all series.
    """
    return prices.dropna() / prices.dropna().iloc[0] * base

In [ ]:
#| eval: false
prices = download_prices('SPY',  period="5y")
rebase(prices, 100).head(10)

Date
2017-09-29    100.000000
2017-10-02    100.433870
2017-10-03    100.648790
2017-10-04    100.768223
2017-10-05    101.365275
2017-10-06    101.249840
2017-10-09    101.082674
2017-10-10    101.349338
2017-10-11    101.508575
2017-10-12    101.357307
Name: Close, dtype: float64

In [ ]:
#| export
def group_returns(returns, groupby, compounded=True):
    """Summarize returns
    group_returns(df, df.index.year)
    group_returns(df, [df.index.year, df.index.month])
    """
    if compounded:
        return returns.groupby(groupby).apply(stats.comp)
    return returns.groupby(groupby).sum()

In [ ]:
#| eval: false
returns = download_returns('SPY',  period="1y")
group_returns(returns, [returns.index.year,returns.index.month])

Date  Date
2021  9      -0.012222
      10      0.070163
      11     -0.008035
      12      0.046248
2022  1      -0.052741
      2      -0.029517
      3       0.037590
      4      -0.087769
      5       0.002257
      6      -0.082460
      7       0.092087
      8      -0.040802
      9      -0.058525
Name: Close, dtype: float64

Note that for year and month, it creates a multicolumns index

In [ ]:
#| export
def aggregate_returns(returns, period=None, compounded=True):
    """Aggregates returns based on date periods, and flattens index"""
    if period is None or 'day' in period:
        return returns
    index = returns.index

    if 'month' in period:
        return group_returns(returns, index.month, compounded=compounded)

    if 'quarter' in period:
        return group_returns(returns, index.quarter, compounded=compounded)

    if period == "A" or any(x in period for x in ['year', 'eoy', 'yoy']):
        return group_returns(returns, index.year, compounded=compounded)

    if 'week' in period:
        return group_returns(returns, index.week, compounded=compounded)

    if 'eow' in period or period == "W":
        grouped = group_returns(returns, [index.year, index.week],
                             compounded=compounded)
        grouped.index = grouped.index.to_flat_index()
        return grouped

    if 'eom' in period or period == "M":
        grouped = group_returns(returns, [index.year, index.month],
                             compounded=compounded)
        grouped.index = grouped.index.to_flat_index()
        return grouped

    if 'eoq' in period or period == "Q":
        grouped = group_returns(returns, [index.year, index.quarterh],
                             compounded=compounded)
        grouped.index = grouped.index.to_flat_index()
        return grouped

    if not isinstance(period, str):
        return group_returns(returns, period, compounded)

    return returns

In [ ]:
#| eval: false
returns = download_returns('SPY',  period="1y")
aggregate_returns(returns, 'M')

(2021, 9)    -0.012222
(2021, 10)    0.070164
(2021, 11)   -0.008035
(2021, 12)    0.046248
(2022, 1)    -0.052741
(2022, 2)    -0.029517
(2022, 3)     0.037590
(2022, 4)    -0.087769
(2022, 5)     0.002257
(2022, 6)    -0.082460
(2022, 7)     0.092087
(2022, 8)    -0.040802
(2022, 9)    -0.058525
Name: Close, dtype: float64

In [ ]:
#| export
def prepare_prices(data, base=1.):
    """Converts return data into prices + cleanup"""
    data = data.copy()
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            if data[col].dropna().min() <= 0 or data[col].dropna().max() < 1:
                data[col] = to_prices(data[col], base)

    # is it returns?
    # elif data.min() < 0 and data.max() < 1:
    elif data.min() < 0 or data.max() < 1:
        data = to_prices(data, base)

    if isinstance(data, (pd.DataFrame, pd.Series)):
        data = data.fillna(0).replace(
            [np.inf, -np.inf], float('NaN'))

    return data

In [ ]:
#| export
def round_to_closest(val, res, decimals=None):
    """Round to closest resolution"""
    if decimals is None and "." in str(res):
        decimals = len(str(res).split('.')[1])
    return round(round(val / res) * res, decimals)

In [ ]:
#| export
def file_stream():
    """Returns a file stream"""
    return io.BytesIO()

In [ ]:
#| export
def _in_notebook(matplotlib_inline=False):
    """Identify enviroment (notebook, terminal, etc)"""
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # Jupyter notebook or qtconsole
            if matplotlib_inline:
                get_ipython().magic("matplotlib inline")
            return True
        if shell == 'TerminalInteractiveShell':
            # Terminal running IPython
            return False
        # Other type (?)
        return False
    except NameError:
        # Probably standard Python interpreter
        return False

In [ ]:
#| export
def _count_consecutive(data):
    """Counts consecutive data (like cumsum() with reset on zeroes)"""
    def _count(data):
        return data * (data.groupby(
            (data != data.shift(1)).cumsum()).cumcount() + 1)

    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            data[col] = _count(data[col])
        return data
    return _count(data)

In [ ]:
#| export
def _score_str(val):
    """Returns + sign for positive values and - for negative values (used in plots)"""
    return ("" if "-" in val else "+") + str(val)

In [ ]:
#| export
def make_index(ticker_weights:(dict), #  A python dict with tickers as keys and weights as values
rebalance="1M", # Pandas resample interval or None for never
period:(str)="max", # time period of the returns to be downloaded
returns:Union['pd.Series', 'pd.DataFrame']=None, #Returns: If provided, check if returns for the ticker are in this dataframe, before trying to download from yahoo
match_dates:(bool)=False # whether to match dates?
)->Union['pd.Series', 'pd.DataFrame']:#Returns for the index
    """
    Makes an index out of the given tickers and weights.
    Optionally you can pass a dataframe with the returns.
    If returns is not given it try to download them with yfinance
    """
    # Declare a returns variable
    index = None
    portfolio = {}

    # Iterate over weights
    for ticker in ticker_weights.keys():
        if (returns is None) or (ticker not in returns.columns):
            # Download the returns for this ticker, e.g. GOOG
            ticker_returns = download_returns(ticker, period)
        else:
            ticker_returns = returns[ticker]

        portfolio[ticker] = ticker_returns

    # index members time-series
    index = pd.DataFrame(portfolio).dropna()

    if match_dates:
        index=index[max(index.ne(0).idxmax()):]

    # no rebalance?
    if rebalance is None:
        for ticker, weight in ticker_weights.items():
            index[ticker] = weight * index[ticker]
        return index.sum(axis=1)

    last_day = index.index[-1]

    # rebalance marker
    rbdf = index.resample(rebalance).first()
    rbdf['break'] = rbdf.index.strftime('%s')

    # index returns with rebalance markers
    index = pd.concat([index, rbdf['break']], axis=1)

    # mark first day day
    index['first_day'] = pd.isna(index['break']) & ~pd.isna(index['break'].shift(1))
    index.loc[index.index[0], 'first_day'] = True

    # multiply first day of each rebalance period by the weight
    for ticker, weight in ticker_weights.items():
        index[ticker] = np.where(
            index['first_day'], weight * index[ticker], index[ticker])

    # drop first marker
    index.drop(columns=['first_day'], inplace=True)

    # drop when all are NaN
    index.dropna(how="all", inplace=True)
    return index[index.index <= last_day].sum(axis=1)


In [ ]:
make_index({'SPY':0.5, 'QQQ':0.5}, period = '5y').tail(10)

C:\Users\kirpa\AppData\Local\Temp\ipykernel_40864\3101533360.py:62: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return index[index.index <= last_day].sum(axis=1)


Date
2022-09-15   -0.028007
2022-09-16   -0.013744
2022-09-19    0.015542
2022-09-20   -0.019450
2022-09-21   -0.035350
2022-09-22   -0.020708
2022-09-23   -0.033036
2022-09-26   -0.014031
2022-09-27   -0.002152
2022-09-28    0.039568
dtype: float64

In [ ]:
#| export
def make_portfolio(returns, start_balance=1e5,
                   mode="comp", round_to=None):
    """Calculates compounded value of portfolio"""
    returns = prepare_returns(returns)

    if mode.lower() in ["cumsum", "sum"]:
        p1 = start_balance + start_balance * returns.cumsum()
    elif mode.lower() in ["compsum", "comp"]:
        p1 = to_prices(returns, start_balance)
    else:
        # fixed amount every day
        comp_rev = (start_balance + start_balance *
                    returns.shift(1)).fillna(start_balance) * returns
        p1 = start_balance + comp_rev.cumsum()

    # add day before with starting balance
    p0 = pd.Series(data=start_balance,
                    index=p1.index + pd.Timedelta(days=-1))[:1]

    portfolio = pd.concat([p0, p1])

    if isinstance(returns, pd.DataFrame):
        portfolio.loc[:1, :] = start_balance
        portfolio.drop(columns=[0], inplace=True)

    if round_to:
        portfolio = np.round(portfolio, round_to)

    return portfolio

In [ ]:
#| export
def _flatten_dataframe(df, set_index=None):
    """Dirty method for flattening multi-index dataframe"""
    s_buf = io.StringIO()
    df.to_csv(s_buf)
    s_buf.seek(0)

    df = pd.read_csv(s_buf)
    if set_index is not None:
        df.set_index(set_index, inplace=True)

    return 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()